<a href="https://colab.research.google.com/github/cgeof/DLC-3-tadpoles-8-labels/blob/main/270123_maDLC_TrainNetwork_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2.2+ Toolbox - COLAB
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1628180434489-T0RIWEJJU0FJVOT6FNVD/maDLC.png?format=800w)

https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- create a multi-animal training set
- train a network
- evaluate a network
- analyze novel videos
- assemble animals and tracklets
- create quality check plots!

###This notebook assumes you already have a DLC project folder with labeled data and you uploaded it to your own Google Drive.

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the docs on GitHub. We also recommend checking out our preprint, which covers the science of maDLC

**Lauer et al 2021:** https://www.biorxiv.org/content/10.1101/2021.04.30.442096v1




## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [ ]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut==2.2.3
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#a few colab specific things needed:
!pip install --upgrade scikit-image
!pip3 install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


After the package is installed, please click "restart runtime" if it appears for DLC changes to take effect in your COLAB environment. You will see this button at the output of the cells above ^.

In [ ]:
import deeplabcut
import pickle5 as pickle

In [ ]:
deeplabcut.__version__


'2.2.3'

## Link your Google Drive (with your labeled data):

- This code assumes you locally installed DeepLabCut, created a project, extracted and labeled frames. Be sure to "check Labels" to confirm you are happy with your data. As, these frames are the only thing that is used to train your network. 💪 You can find all the docs to do this here: https://deeplabcut.github.io/DeepLabCut

- Next, place your DLC project folder into you Google Drive- i.e., copy the folder named "Project-YourName-TheDate" into Google Drive.

- Then, click run on the cell below to link this notebook to your Google Drive:

In [ ]:
#Now, let's link to your Google Drive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Next, edit the few items below, and click run:


In [ ]:
# PLEASE EDIT THIS:
ProjectFolderName = '2022-06-10-labo-cg-2022-09-27'
VideoType = 'mp4' #, mp4, MOV, or avi, whatever you uploaded!


# No need to edit this, we are going to assume you put videos you want to analyze in the "videos" folder, but if this is NOT true, edit below:
videofile_path = ['/content/drive/My Drive/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.
videofile_path

#No need to edit this, as you set it when you passed the ProjectFolderName (above): 
path_config_file = '/content/drive/My Drive/'+ProjectFolderName+'/config.yaml'
path_config_file
#This creates a path variable that links to your google drive project

'/content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/config.yaml'

## Create a multi-animal training dataset:

- more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#create-training-dataset

- please check the text below, edit if needed, and then click run (this can take some time):

In [ ]:
#OPTIONAL LEARNING: did you know you can check what each function does by running with a ?
deeplabcut.create_multianimaltraining_dataset?

In [ ]:
# ATTENTION:
#which shuffle do you want to create and train?
shuffle = 11 #edit if needed; 1 is the default.

#if you labeled on Windows, please set the windows2linux=True:
#deeplabcut.create_multianimaltraining_dataset(path_config_file, Shuffles=[shuffle], net_type="dlcrnet_ms5",windows2linux=False)

## Start training:
This function trains the network for a specific shuffle of the training dataset. 
 - more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#train-the-network

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#Typically, you want to train to 50,000 - 200K iterations.
#more info and there are more things you can set: https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=100,saveiters=1000, maxiters=75000, allow_growth=True)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 50K iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

Selecting multi-animal trainer


Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7]],
 'all_joints_names': ['Nose',
                      'R_Eye',
                      'L_eye',
                      'Stomach',
                      'T1',
                      'T2',
                      'T3',
                      'T4'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 8,
 'contrast': {'clahe': True,
              'claheratio': 0.1,
              'histeq': True,
              'histeqratio': 0.1},
 'convolution': {'edge': False,
                 'emboss': {'alpha': [0.0, 1.0], 'strength': [0.5, 1.5]},
                 'embossratio': 0.1,
                 'sharpen': False,
                 'sharpenratio': 0.3},
 'crop_pad': 0,
 'crop_sampling': 'hybrid',
 'crop_size': [400, 400],
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_2022-06-10-laboSep27/2022-06-10-labo_cg95shuffle11.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'decay_steps': 30000,
 'deterministic': False

Activating limb prediction...
Batch Size is 8
Getting specs multi-animal-imgaug 28 8


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:684: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  outputs = layer.apply(inputs, training=is_training)


Loading already trained DLC with backbone: resnet_50
Max_iters overwritten as 75000
Display_iters overwritten as 100
Save_iters overwritten as 1000
Training parameters:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/dlc-models/iteration-0/2022-06-10-laboSep27-trainset95shuffle11/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'adam', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 8, 'dataset_type': 'multi-animal-imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': True

iteration: 168100 loss: 0.0053 scmap loss: 0.0049 locref loss: 0.0001 limb loss: 0.0003 lr: 0.0001
iteration: 168200 loss: 0.0054 scmap loss: 0.0050 locref loss: 0.0001 limb loss: 0.0003 lr: 0.0001
iteration: 168300 loss: 0.0059 scmap loss: 0.0055 locref loss: 0.0001 limb loss: 0.0003 lr: 0.0001
iteration: 168400 loss: 0.0058 scmap loss: 0.0054 locref loss: 0.0001 limb loss: 0.0003 lr: 0.0001
iteration: 168500 loss: 0.0058 scmap loss: 0.0054 locref loss: 0.0001 limb loss: 0.0003 lr: 0.0001
iteration: 168600 loss: 0.0060 scmap loss: 0.0056 locref loss: 0.0001 limb loss: 0.0003 lr: 0.0001
iteration: 168700 loss: 0.0060 scmap loss: 0.0056 locref loss: 0.0001 limb loss: 0.0003 lr: 0.0001
iteration: 168800 loss: 0.0057 scmap loss: 0.0053 locref loss: 0.0001 limb loss: 0.0003 lr: 0.0001
iteration: 168900 loss: 0.0061 scmap loss: 0.0057 locref loss: 0.0001 limb loss: 0.0003 lr: 0.0001
iteration: 169000 loss: 0.0061 scmap loss: 0.0056 locref loss: 0.0001 limb loss: 0.0004 lr: 0.0001
iteration:

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating: 

 - First, we evaluate the pose estimation performance.

- This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, don't proceed to tracklet assembly; please consider (1) adding more data, (2) adding more bodyparts!

- more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#evaluate-the-trained-network

Here is an example of what you'd aim to see before proceeding:

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590535809087-X655WY9W1MW1MY1I7DHE/ke17ZwdGBToddI8pDm48kBoswZhKnUtAF7-bTXgw67EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKc5tTP1cnANTUwNNPnYFjIp6XbP9N1GxIgAkxvBVqt0UvLpPHYwvNQTwHg8f_Zu8ZF/evaluation.png?format=1000w)



In [ ]:
#let's evaluate first:
deeplabcut.evaluate_network(path_config_file,Shuffles=[shuffle], plotting=True)
#plot a few scoremaps:
deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0])

Running  DLC_dlcrnetms5_2022-06-10-laboSep27shuffle11_243000  with # of trainingiterations: 243000


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:684: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  outputs = layer.apply(inputs, training=is_training)


Activating extracting of PAFs
Network Evaluation underway...


1673it [14:11,  1.97it/s]


Results for 243000 training iterations, training fraction of 95, and shuffle 11:
Train error: 2.72 pixels. Test error: 4.85 pixels.
With pcutoff of 0.4:
Train error: 2.7 pixels. Test error: 4.78 pixels.
##########################################
Average Euclidean distance to GT per individual (in pixels; test-only)
individuals
tadpole1    4.373090
tadpole2    5.129354
tadpole3    5.028751
Average Euclidean distance to GT per bodypart (in pixels; test-only)
bodyparts
L_eye       3.114443
Nose        3.381510
R_Eye       3.586267
Stomach     3.090311
T1          3.458280
T2          5.102679
T3          7.978693
T4         10.311673
Done and results stored for snapshot:  snapshot-243000
Selecting best skeleton...
Graph 1|10


...


100%|██████████| 1672/1672 [00:00<00:00, 2305.69it/s]


Activating extracting of PAFs
Analyzing data...


1it [00:00,  3.23it/s]


Saving plots...


100%|██████████| 1/1 [00:02<00:00,  2.75s/it]


IF these images, numbers, and maps do not look good, do not proceed. You should increase the diversity and number of frames you label, and re-create a training dataset and re-train! 

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in a pickle file in the same directory where the video resides. 

In [ ]:
print("Start Analyzing my video(s)!")
#EDIT OPTION: which video(s) do you want to analyze? You can pass a path or a folder:
# currently, if you run "as is" it assumes you have a video in the DLC project video folder!

deeplabcut.analyze_videos(path_config_file,videofile_path, shuffle=shuffle, videotype=VideoType)

Start Analyzing my video(s)!
Using snapshot-243000 for model /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/dlc-models/iteration-0/2022-06-10-laboSep27-trainset95shuffle11


/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Activating extracting of PAFs
Analyzing all the videos in the directory...
Starting to analyze %  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221216_144956552_cropped-2 (1).mp4
Loading  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221216_144956552_cropped-2 (1).mp4
Duration of video [s]:  60.0 , recorded with  25.0 fps!
Overall # of frames:  1500  found with (before cropping) frame dimensions:  354 364
Starting to extract posture from the video(s) with batchsize: 8


 16%|█▌        | 240/1500 [02:32<13:04,  1.61it/s]

Optional: Now you have the option to check the raw dections before animals are assembled. To do so, pass a video path:

In [ ]:
##### PROTIP: #####
## look at the output video; if the pose estimation (i.e. key points)
## don't look good, don't proceed with tracking - add more data to your training set and re-train!

#EDIT: let's check a specific video (PLEASE EDIT VIDEO PATH):
Specific_videofile = '/content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_131537186_cropped-2.mp4'


#don't edit:
deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], shuffle=shuffle)

Creating labeled video for  Basler_acA800-510um__23617456__20221214_131537186_cropped-2


100%|██████████| 1285/1285 [00:04<00:00, 267.41it/s]


If the resutling video (ends in full.mp4) is not good, we highly recommend adding more data and training again. See here: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#decision-break-point

# Next, we will assemble animals using our data-driven optimal graph method:

- Here, we will find the optimal graph, which matches the "data-driven" method from our paper (Figure adapted from Lauer et al. 2021):

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1626266017809-XO6NX84QB4FBAZGOTCEY/fig3.jpg?format=400w)


- note, you can set the number of animals you expect to see, so check, edit, then click run:

In [ ]:
#Check and edit:
numAnimals = 3 #how many animals do you expect to find?
tracktype= 'ellipse' #box, skeleton, ellipse:
#-- ellipse is recommended, unless you have a single-point ma project, then use BOX!

#Optional: 
#imagine you tracked a point that is not useful for assembly, 
#like a tail tip that is far from the body, consider dropping it for this step (it's still used later)!
#To drop it, uncomment the next line TWO lines and add your parts(s):

bodypart= 'T4'
deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, overwrite=True, ignore_bodyparts=[bodypart])

#OR don't drop, just click RUN:
#deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, 
                                        #shuffle=shuffle, overwrite=True)

deeplabcut.stitch_tracklets(path_config_file, videofile_path, shuffle=shuffle, track_method=tracktype, n_tracks=numAnimals)

Using snapshot-243000 for model /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/dlc-models/iteration-0/2022-06-10-laboSep27-trainset95shuffle11
Analyzing all the videos in the directory...
Processing...  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_174431546_cropped-1.mp4
Analyzing /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_174431546_cropped-1DLC_dlcrnetms5_2022-06-10-laboSep27shuffle11_243000.h5


100%|██████████| 1500/1500 [00:12<00:00, 116.86it/s]
1500it [00:13, 112.44it/s]


Processing...  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_173413406_cropped-1.mp4
Analyzing /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_173413406_cropped-1DLC_dlcrnetms5_2022-06-10-laboSep27shuffle11_243000.h5


100%|██████████| 4376/4376 [00:13<00:00, 327.29it/s]
4376it [00:04, 890.27it/s]


Processing...  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_150147670_cropped-2.mp4
Analyzing /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_150147670_cropped-2DLC_dlcrnetms5_2022-06-10-laboSep27shuffle11_243000.h5


...

100%|██████████| 79/79 [00:00<00:00, 2938.59it/s]


Processing...  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_125423841_cropped-1.mp4


100%|██████████| 9/9 [00:00<00:00, 1585.28it/s]


Processing...  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_112658018_cropped-1.mp4


100%|██████████| 3/3 [00:00<00:00, 14997.51it/s]


Now let's filter the data to remove any small jitter:

In [ ]:
deeplabcut.filterpredictions(path_config_file, 
                                 videofile_path, 
                                 shuffle=shuffle,
                                 videotype=VideoType, 
                                 track_method = tracktype)

Analyzing all the videos in the directory...
Filtering with median model /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_170308103_cropped-1.mp4
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_164613164_cropped-1.mp4
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_172836792_cropped-1.mp4
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_151136466_cropped-2.mp4
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221214_154527326_cropped-1.mp4
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/202

## Create plots of your trajectories:

In [ ]:
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, track_method=tracktype)

Analyzing all the videos in the directory...
Loading  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221026_170937653_cropped-1.mp4 and data.
Loading  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221026_172124290_cropped-1.mp4 and data.
Loading  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221026_172018032_cropped-2.mp4 and data.
Loading  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221026_160601671_cropped-2.mp4 and data.
Loading  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221026_171115828_cropped-1.mp4 and data.
Loading  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221026_171853463_cropped-1.mp4 and data.
Loading  /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path, 
                                shuffle=shuffle, 
                                color_by="individual",
                                videotype=VideoType, 
                                save_frames=True,
                                filtered=True)

Analyzing all the videos in the directory...
Starting to process video: /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221020_153247995_cropped-1.mp4Starting to process video: /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221020_153353997_cropped-1.mp4

Loading /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221020_153353997_cropped-1.mp4 and data.
Loading /content/drive/My Drive/2022-06-10-labo-cg-2022-09-27/videos/Basler_acA800-510um__23617456__20221020_153247995_cropped-1.mp4 and data.
Duration of video [s]: 60.0, recorded with 25.0 fps!
Overall # of frames: 1500 with cropped frame dimensions: 328 348
Generating frames and creating video.
Duration of video [s]: 175.04, recorded with 25.0 fps!
Overall # of frames: 4376 with cropped frame dimensions: 336 340
Generating frames and creating video.


  3%|▎         | 133/4376 [00:59<31:24,  2.25it/s]


KeyboardInterrupt: ignored